# Restaurant Review Classification Based on 3 Aspect

Micheila Jiemesha - 0706012110032
<br>Marsha Alexis Likorawung - 0706012110034
<br>Michelle Swastika Bianglala Nusantara - 0706012110002
<br>Rifqie Tilqa Reamizard - 0706012110025

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('restaurant_reviews_labelled.csv')

In [3]:
import spacy
import re
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))

    # Clean the text: remove non-word characters, convert to lowercase, and handle extra spaces
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)

    # Remove stopwords and lemmatize the remaining words
    text = ' '.join([token.lemma_ for token in nlp(" ".join([word for word in text.split() if word not in stop_words]))])

    return text

In [4]:
data['Review'] = data['Review'].apply(preprocess_text)

In [5]:
data.to_csv('restaurant_reviews_preprocessed.csv', index=False)

In [6]:
targets = ['Food & Drinks Quality & Price', 'General (Ambience, Entertainment, & Experience)', 'Service']

for target in targets:
    data[target] = data[target].fillna('Good')

In [7]:
val_data = data.sample(n=36, random_state=42)

In [8]:
data = data.drop(val_data.index)

In [9]:
data[targets[0]].value_counts()

Food & Drinks Quality & Price
Good    3903
Bad     1297
Name: count, dtype: int64

In [10]:
data[targets[1]].value_counts()

General (Ambience, Entertainment, & Experience)
Good    4119
Bad     1081
Name: count, dtype: int64

In [11]:
data[targets[2]].value_counts()

Service
Good    4115
Bad     1085
Name: count, dtype: int64

In [12]:
datasets = {}

for target in targets:
    target_data = data[['Review', target]]
    datasets[target] = target_data

In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import joblib

def train_and_evaluate_model(datasets, model_name, model_class, **model_params):
    models = {}

    for target, dataset in datasets.items():
        X = dataset['Review']
        y = dataset[target].apply(lambda x: 1 if x == 'Good' else 0)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        vectorizer = TfidfVectorizer(max_features=5000)
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)

        # Modify SMOTE for multi-class
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

        model = model_class(**model_params)
        model.fit(X_train_resampled, y_train_resampled)

        y_pred = model.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'Accuracy for {target}: {accuracy:.2f}')
        print(f'Performance for {target}:')
        print(classification_report(y_test, y_pred))

        joblib.dump(model, f'{model_name}_{target}_model.pkl')
        joblib.dump(vectorizer, f'{model_name}_{target}_vectorizer.pkl')
        models[target] = (model, vectorizer)
        print()

    return models

In [14]:
def predict_review(review, models):
    review = preprocess_text(review)
    predictions = {}
    
    for target in models:
        model, vectorizer = models[target]
        
        review_vec = vectorizer.transform([review])
        prediction = model.predict(review_vec)[0]
        
        predictions[target] = 'Good' if prediction == 1 else 'Bad'
        
    return predictions

In [15]:
def evaluate_val_data(val_data, models, target_columns):
    accuracies = {}

    for target in target_columns:
        true_labels = val_data[target].map({'Good': 1, 'Bad': 0})
        predictions = []

        for review in val_data['Review']:
            review_predictions = predict_review(review, {target: models[target]})
            predicted_label = 1 if review_predictions[target] == 'Good' else 0
            predictions.append(predicted_label)

        accuracy = accuracy_score(true_labels, predictions)
        accuracies[target] = accuracy
        print(f"Validation Accuracy for {target}: {accuracy:.2f}")

    return accuracies

## Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression

lr_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='LogisticRegression',
    model_class=LogisticRegression,
    max_iter=1000,
    random_state=42
)

Accuracy for Food & Drinks Quality & Price: 0.84
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.65      0.77      0.70       248
           1       0.92      0.87      0.89       792

    accuracy                           0.84      1040
   macro avg       0.78      0.82      0.80      1040
weighted avg       0.86      0.84      0.85      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.86
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.65      0.75      0.70       216
           1       0.93      0.89      0.91       824

    accuracy                           0.86      1040
   macro avg       0.79      0.82      0.80      1040
weighted avg       0.87      0.86      0.87      1040


Accuracy for Service: 0.89
Performance for Service:
              precision    recall  f1-score   support

         

In [17]:
lr_accuracies = evaluate_val_data(val_data, lr_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.78
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.89
Validation Accuracy for Service: 0.92


## Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier

rf_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='RandomForest',
    model_class=RandomForestClassifier,
    n_estimators=100,
    random_state=42
)

Accuracy for Food & Drinks Quality & Price: 0.82
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.63      0.58      0.61       248
           1       0.87      0.89      0.88       792

    accuracy                           0.82      1040
   macro avg       0.75      0.74      0.74      1040
weighted avg       0.81      0.82      0.82      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.83
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.63      0.44      0.52       216
           1       0.86      0.93      0.90       824

    accuracy                           0.83      1040
   macro avg       0.75      0.69      0.71      1040
weighted avg       0.82      0.83      0.82      1040


Accuracy for Service: 0.88
Performance for Service:
              precision    recall  f1-score   support

         

In [19]:
rf_accuracies = evaluate_val_data(val_data, rf_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.86
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.72
Validation Accuracy for Service: 0.86


## Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier

dt_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='DecisionTree',
    model_class=DecisionTreeClassifier,
    max_depth=None,
    random_state=42
)

Accuracy for Food & Drinks Quality & Price: 0.77
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.52      0.56      0.54       248
           1       0.86      0.84      0.85       792

    accuracy                           0.77      1040
   macro avg       0.69      0.70      0.69      1040
weighted avg       0.78      0.77      0.78      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.77
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.45      0.50      0.47       216
           1       0.87      0.84      0.85       824

    accuracy                           0.77      1040
   macro avg       0.66      0.67      0.66      1040
weighted avg       0.78      0.77      0.77      1040


Accuracy for Service: 0.82
Performance for Service:
              precision    recall  f1-score   support

         

In [21]:
dt_accuracies = evaluate_val_data(val_data, dt_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.75
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.81
Validation Accuracy for Service: 0.78


## KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='KNN',
    model_class=KNeighborsClassifier,
    n_neighbors=10
)

Accuracy for Food & Drinks Quality & Price: 0.27
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.25      1.00      0.40       248
           1       1.00      0.04      0.08       792

    accuracy                           0.27      1040
   macro avg       0.62      0.52      0.24      1040
weighted avg       0.82      0.27      0.16      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.24
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.21      1.00      0.35       216
           1       1.00      0.04      0.07       824

    accuracy                           0.24      1040
   macro avg       0.61      0.52      0.21      1040
weighted avg       0.84      0.24      0.13      1040


Accuracy for Service: 0.22
Performance for Service:
              precision    recall  f1-score   support

         

In [23]:
knn_accuracies = evaluate_val_data(val_data, knn_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.39
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.36
Validation Accuracy for Service: 0.28


## XGBoost

In [24]:
from xgboost import XGBClassifier

xgb_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='XGB',
    model_class=XGBClassifier,
    random_state=42
)

Accuracy for Food & Drinks Quality & Price: 0.85
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.71      0.65      0.68       248
           1       0.89      0.92      0.90       792

    accuracy                           0.85      1040
   macro avg       0.80      0.78      0.79      1040
weighted avg       0.85      0.85      0.85      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.85
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.66      0.57      0.61       216
           1       0.89      0.92      0.91       824

    accuracy                           0.85      1040
   macro avg       0.78      0.75      0.76      1040
weighted avg       0.84      0.85      0.85      1040


Accuracy for Service: 0.89
Performance for Service:
              precision    recall  f1-score   support

         

In [25]:
xgb_accuracies = evaluate_val_data(val_data, xgb_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.83
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.83
Validation Accuracy for Service: 0.86


## LightGBM

In [26]:
from lightgbm import LGBMClassifier

lgbm_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='LGBM',
    model_class=LGBMClassifier,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

[LightGBM] [Info] Number of positive: 3111, number of negative: 3111
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.066740 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 93974
[LightGBM] [Info] Number of data points in the train set: 6222, number of used features: 1973
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy for Food & Drinks Quality & Price: 0.86
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       248
           1       0.90      0.91      0.91       792

    accuracy                           0.86      1040
   macro avg       0.80      0.79      0.80      1040
weighted avg       0.85      0.86      0.85      1040


[LightGBM] [Info] Number of positive: 3295, number of negative: 3295
[Lig

In [27]:
lgbm_accuracies = evaluate_val_data(val_data, lgbm_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.83
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.83
Validation Accuracy for Service: 0.83


## CatBoost

In [28]:
from catboost import CatBoostClassifier

catboost_models = train_and_evaluate_model(
    datasets=datasets,
    model_name='CatBoost',
    model_class=CatBoostClassifier,
    iterations=500,
    learning_rate=0.1,
    depth=6,
    verbose=0,
    random_seed=42
)

Accuracy for Food & Drinks Quality & Price: 0.85
Performance for Food & Drinks Quality & Price:
              precision    recall  f1-score   support

           0       0.70      0.67      0.69       248
           1       0.90      0.91      0.91       792

    accuracy                           0.85      1040
   macro avg       0.80      0.79      0.80      1040
weighted avg       0.85      0.85      0.85      1040


Accuracy for General (Ambience, Entertainment, & Experience): 0.85
Performance for General (Ambience, Entertainment, & Experience):
              precision    recall  f1-score   support

           0       0.68      0.58      0.62       216
           1       0.89      0.93      0.91       824

    accuracy                           0.85      1040
   macro avg       0.78      0.75      0.77      1040
weighted avg       0.85      0.85      0.85      1040


Accuracy for Service: 0.90
Performance for Service:
              precision    recall  f1-score   support

         

In [29]:
catboost_accuracies = evaluate_val_data(val_data, catboost_models, targets)

Validation Accuracy for Food & Drinks Quality & Price: 0.83
Validation Accuracy for General (Ambience, Entertainment, & Experience): 0.83
Validation Accuracy for Service: 0.86
